In [1]:
import pandas as pd
import re

In [2]:
pd.options.display.max_columns = 20

In [16]:
# df: 0530합본_new 전처리 하기 전 데이터 셋인듯 -> 명세서 정보
df = pd.read_csv("0428sum_taxdata_cp949.csv", encoding = 'cp949', index_col=0)

# df_s: invoicer, invoicee 정보
df_s = pd.read_csv("0503_raw_data.csv", encoding="utf-8") 

# df에서 두번째 list에 있는 column만 추출
df_ = df.loc[:, ['상호', '업태',
       '종목', '일자', '발급일자', '전송일자','거래처', '품명', '공급가액', '세액',
       '상대계정', '승인번호']]

/home/r320/anaconda3/envs/nlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (1,8,9,10,12,13,14,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
# pd.merge: 각 데이터에 존재하는 고유값(key)을 기준으로 병합할 때 사용
# df_의 승인번호 ==  df_s의 issueID -> 이 key값을 기준으로 같은 row끼리 병합
new_df = pd.merge(left=df_, right = df_s, how='left', left_on='승인번호', right_on = 'issueID')

# issueID column에 null인 곳은 True로 아닌 곳은 false로 변환하고 이 df를 add_df에 저장
add_df = new_df[new_df['issueID'].isnull()] 

In [21]:
print(len(df['상대계정'].unique()))

90


In [8]:
# invoice: 거래상품명세서
# 명세서 발급 받은 사람 정보: invoicee_info
invoicee_info = df_s[['invoiceeID', 'invoiceeNametext',
       'invoiceeSpecifiedPerson', 'invoiceeLineOneText', 'invoiceeTypeCode',
       'invoiceeClassificationCode']]
# 'invoiceeID' column값이 중복인 것들을 제거
invoicee_info = invoicee_info.drop_duplicates(subset=['invoiceeID']) 
invoicee_info = invoicee_info.dropna()

# 명세서 발행인 정보: invoicer_info
# 'invoicerID' column값이 중복인 것들을 제거
invoicer_info = df_s[['invoicerID', 'invoicerNametext',
       'invoicerSpecifiedPerson', 'invoicerLineOneText', 'invoicerTypeCode',
       'invoicerClassificationCode']]
invoicer_info = invoicer_info.drop_duplicates(subset=['invoicerID'])
invoicer_info = invoicer_info.dropna()

invoicee_info.columns = ['ID', 'Nametext',
       'SpecifiedPerson', 'LineOneText', 'TypeCode',
       'ClassificationCode']
invoicer_info.columns = ['ID', 'Nametext',
       'SpecifiedPerson', 'LineOneText', 'TypeCode',
       'ClassificationCode']

# 정수 index를 가지고 있는 명세서 발급자, 명세서 발급받은 사람 정보를 담은 info pd를 생성
# 두 df를 위아래로 병합하고 회사명('Nametext')이 중복되는 row는 제거
# 최종적으로 상품명세서를 발급하거나 발급받은 회사에 대한 정보만 담고 있음
info = pd.concat([invoicer_info, invoicee_info]).reset_index().iloc[:,1:]
info = info.drop_duplicates(subset=['Nametext'])

In [9]:
info_add = pd.DataFrame([[2418500724,
 '세무법인동안(서초)',
 '정용식',
 '서울특별시 서초구 서초대로26길 3   4층401호(방배동)',
 '서비스(전문/과학및기술)업',
 '세무사'],
[2418500724, '세무법인 동안', '정용식', '서울특별시 서초구 동광로 82  4층 1호(방배동)', '전문서비스업', '세무사업'],
[1028142945,
 '(주)케이티',
 '구현모',
 '경기도 성남시 분당구 불정로 90',
 '통신업 부동산업',
 '전신전화 부가통신 임대'],
[1028142945,
 '(주)케이티 ',
 '구현모',
 '경기도 성남시 분당구 불정로 90',
 '통신업 부동산업',
 '전신전화 부가통신 임대'],
 [1028142945,
 '주식회사케이티',
 '구현모',
 '경기도 성남시 분당구 불정로 90',
 '통신업 부동산업',
 '전신전화 부가통신 임대'],
[2208139938,
 '（주）엘지유플러스',
 '하현회',
 '서울특별시 용산구 한강대로 32(한강로3가  엘지유플러스)',
 '출판 영상 방송통신 및 정보서비스업',
 '유선통신업'],
[2208101733,
 '다우데이타',
 '이인복',
 '서울특별시 마포구 독막로 311 (염리동  재화스퀘어)5  11층',
 '서비스 외',
 'S/W H/W개발공급자문임대 외'],
[1318624222,
 '(주)금송로지스틱스',
 '김영희',
 '경기도 평택시 포승읍 평택항만길 86103호',
 '서비스',
 '복합운송주선업'],
[0, 'OK주유소', '', '', '도매 및 소매업', '주유소'],
[2208794836, '（주）인애', '안재진', '서울특별시 서초구 고무래로 6-5(반포동)', '서비스', '광고대행'],
[0,
 '(주)에스원 아산',
 ' ',
 ' ',
 '통신  사업서비스업',
 '전기경보및 신호  경비'],
 [0,
 '(주)에스원 서강',
 ' ',
 ' ',
 '통신  사업서비스업',
 '전기경보및 신호  경비'],
  [0,
 '(주)에스원 본사',
 ' ',
 ' ',
 '통신  사업서비스업',
 '전기경보및 신호  경비'],
  [0,
 '(주)에스원 경기광주',
 ' ',
 ' ',
 '통신  사업서비스업',
 '전기경보및 신호  경비'],
    [0,
 '(주)에스원 부천',
 ' ',
 ' ',
 '통신  사업서비스업',
 '전기경보및 신호  경비'],
[1408504994,
 '(주)에이디티캡스시흥',
 '최진환',
 '경기도 시흥시 옥구상가1길 15-1 청한빌딩 5층 (정왕동)',
 '서비스',
 '무인경비외'],
[1068635117,
 '（주）마인드온 ',
 '이종학',
 '서울특별시 용산구 원효로 152-1 (원효로3가) 원기빌딩 202호',
 '도소매',
 '소프트웨어외'],
[6068188569,
 '(주)디모아',
 '이혁수 이장훈',
 '서울시 강남구 언주로 726 11층 (논현동 두산빌딩)',
 '서비스  도소매',
 'S/W H/W개발및공급'],
[2208742885, '(주)야놀자(본사)', '이수진', '서울시 강남구 테헤란로 108길 42', '서비스', '온라인정보제공업'],
                    [0,
 '형제전기(주) - 시화',
 ' ',
 ' ',
 '제조 도매',
 '전기자재'],
                    [1258506450,
 '㈜에이디티캡스평택',
 '박진효',
 '경기도 평택시 서재로 7  3층(동삭동  주영빌딩)',
 '서비스',
 '무인경비외'],
                    [5428501005,
 '(주)에이디티캡스 금천지점',
 '박진효',
 '서울특별시 금천구 가산디지털1로 181  11층 1101~1104호',
 '서비스',
 '무인경비외'],
                                        [0,
 '(주)에이디티캡스 하남지점',
 ' ',
 ' ',
 '서비스',
 '무인경비외'],
                     [1348203927,
 '(재)경기테크노파크(임대보증금)',
 '배수용',
 '(15588)경기도 안산시 상록구 해안로 705 (사동  경기테크노파크) 지원편의동 228호 시설운영팀',
 '서비스업  부동산임대업',
 '임대  연구용역 외'], 
                      [2048653952,
 '(주)윤건세라믹',
 '노경석',
 '경기도 하남시 샘재로119번길 179(천현동)',
 '도매 및 소매업',
 '위생도기'],[2288700502,
 '유한회사클린산업',
 '김기호',
 '광주광역시 광산구 체암로 1612(양동)',
 '도매 및 소매업',
 '전기자재 철재료및기타자재'],[2721500972, '베짱이노래연습장', 
'이병주', '경기 성남시 중원구 광명로 312 2층 (금광동)', '오락문화등', '노래방'],
                     [1348194528,
 '(주)로드텍',
 '문형철',
 '경기도 안산시 상록구 해안로 705(사동  경기테크노파크 RIT센터 203 204 205호)',
 '제조업',
 '측량기구/도로포장상태조사장비개발/계측기기'],[7528600124, '(주)유니크시스템', 
'주재희', '경기도 포천시 군내면 용정경제로2길 62-0', '건설업외', '창호공사외'],
  [4258700592, '(주)에스엠실업', '전덕자', '서울특별시 동대문구 답십리동 494-5  464-6호', '부동산', '임대'],
    [3148643839,
 '(주)문화유산기술연구소',
 '김지교 외 1 명',
 '대전광역시 중구 계룡로 839(용두동  문화유산기술연구소)',
 '서비스',
 '문화유산디지털복원업'],[1304394015,
 '(주)파이오니크',
 '장인자',
 '경기 광명시 안현로 34 307동 303호 (하안동 하',
 '제조업',
 '시제품(모형)'],[1198144562,
 '호마종합통상',
 '강순동',
 '서울 금천구 시흥대로28길35-6  2층(시흥동 945-36번지 호마빌딩)',
 '도소매 제조 서비스',
 '공작기계 오퍼'],[6798700049, '(주)씨앤아이솔루션', '김은진', '경기도 평택시 진위면 은산길 30', '제조업', '반도체부품가공업'],
                     [5668100964,
 '(주)아이디주택',
 '박성수',
 '서울특별시 강서구 공항대로 46  지층 1호(공항동)',
 '건설업',
 '철근콘크리트공사'],[2208814921,
 '제이케이앤(주)',
 '안재진',
 '서울특별시 서초구 고무래로6-5  (반포동)',
 '서비스 출판 소매',
 '광고  홍보마케팅  신문 외'],
                     [1080153598,
 'DSP(디에스피)',
 '최성수',
 '서울특별시 중구 마른내로6길 29-5(인현동2가  2층)',
 '제조',
 '인쇄'],
                     [1718200026,
 '의료법인 성누가의료재단',
 '이영석',
 '서울특별시 은평구 통일로 726 불광메디컬타워 (불광동)',
 '서비스 도매및소매업',
 '병원 건강보조식품 소매업'],
                     [1068710729,
 '(주)럭스로보',
 '오상훈',
 '서울특별시 강남구 강남대로 311   1306~8 주식회사 럭스로보',
 '제조업',
 '교구및교육용로봇제조업'],[1078163208, '(주)동양보일러뱅크', 
                 '도창교', '경기도 화성시 장안면 석포공단길 2', '제조', '산업보일러'], # 추가
  [0, '주식회사 미드텍', " ", " ", "제조업", "전기자동제어,설비"],
  [0, '주식회사 치즈앤', " ", " ", "정보통신업", "영상물제작외"],
  [0, '주식회사 윤폴리머', " ", " ", "도매 및 상품중개", "플라스틱판제"],
  [0, '(주)제이티에스', " ", " ", "제조", "프레스금형"],
  [0, '(주)거상푸드시스템', " ", " ", "도매,제조", "식자재유통,식자재"],
  [0, '(주)누리시스템스', " ", " ", "서비스(전문/과학)", "중재및공공관계서"],
                         
  [0, '(주)케이원테크', " ", " ", "제조업", "반도체장비제작"],
  [0, '(주)다한축산', " ", " ", "도매 및 상품중개", "축산물(일반)"],
  [0, '주식회사 씨앤세스시스템즈', " ", " ", "도소매,제조,전문과", "전기,전자,반도체건"],
  [0, '꽉찬만두', " ", " ", "음식", "만두"],
  [0, '주식회사 베이글랩스(BagelLabsco.Ltd.)', " ", " ", "제조업,도매및소매", "전자기기및부품개발"],
  [0, '(주)서정툴링', " ", " ", "도매", "기계공구외"],
                         
                        ],

 columns = ['ID', 'Nametext', 'SpecifiedPerson', 'LineOneText', 'TypeCode',
       'ClassificationCode'],)



In [10]:
info_new = pd.concat([info, info_add]) # 새로운 invoicer | invoicee 정보 추가

In [11]:
info_new.head()

,ID,Nametext,SpecifiedPerson,LineOneText,TypeCode,ClassificationCode
0,8808101059,주식회사 모닝상사,김영의,인천 인천 부평구 백범로 577번길 15-12,도소매업 제조업,작업복 안전용품 청소용품 외
1,1028123104,알파색채(주),남궁요숙,서울시 종로구 평창8길 12,제조업 도매,화학용물감 무역
2,1058652618,(주) 앤비젼,김덕표,서울특별시 금천구 가산디지털 2로 98 1-603 (가산동 아이티캐슬),제조 도매 서비스,컴퓨터주변기기 전자부품 SW자문
3,2208139938,㈜LG유플러스,하현회,서울시 용산구 한강대로 32,통신 제조 도매 소매 서비스업,개인휴대통신외
4,1238146502,(주)공진,윤규중 윤지우,경기도 군포시 당정로 10 (당정동),제조업,산업용핸들외


In [12]:
len(info_new)

13660

In [13]:
new_df1 = pd.merge(left=new_df[['상호', '업태', '종목', '일자', '발급일자', '전송일자', '거래처', '품명', '공급가액', '세액',
       '상대계정', '승인번호']], right = info_new, how='left', left_on='거래처', right_on = 'Nametext')

In [17]:
print(len(new_df1))

print(len(info_new))

38138
13660


In [16]:
new_df1['Nametext'].isnull().sum() # Nametext가 null이다 == invoice 정보가 없는 row이다?

14357

In [23]:
len(new_df1)

38138

In [24]:
new_df1.columns

Index(['상호', '업태', '종목', '일자', '발급일자', '전송일자', '거래처', '품명', '공급가액', '세액',
       '상대계정', '승인번호', 'ID', 'Nametext', 'SpecifiedPerson', 'LineOneText',
       'TypeCode', 'ClassificationCode'],
      dtype='object')

### (주) 특수기호 때문에 회사 명이 매치가 안되는 형상 발생한듯. 그래서 특수기호를 바꿔줌

In [28]:
# new_df1 : column 이름 바꿈 , 상호 (주) 특수기호 바꿈

new_df2 = new_df1.copy() # deep copy
# new_df2 = new_df1[['상호', '업태', '종목', '일자', '발급일자', '전송일자', '거래처', '품명', '공급가액', '세액',
#        '상대계정', '승인번호', 'invoicerID', 'invoicerNametext',
#        'invoicerSpecifiedPerson', 'invoicerLineOneText', 'invoicerTypeCode',
#        'invoicerClassificationCode',]].copy()

# new_df2.columns = ['상호', '업태', '종목', '일자', '발급일자', '전송일자', '거래처', '품명', '공급가액', '세액',
#        '상대계정', '승인번호', 'ID', 'Nametext', 'SpecifiedPerson', 'LineOneText',
#        'TypeCode', 'ClassificationCode']

new_df2['상호'] = new_df2['상호'].str.replace("㈜", "(주)")

new_df3 = pd.merge(left=new_df2, right = info_new, how='left', left_on='상호', right_on = 'Nametext')

In [29]:
new_df3.columns

Index(['상호', '업태', '종목', '일자', '발급일자', '전송일자', '거래처', '품명', '공급가액', '세액',
       '상대계정', '승인번호', 'ID_x', 'Nametext_x', 'SpecifiedPerson_x',
       'LineOneText_x', 'TypeCode_x', 'ClassificationCode_x', 'ID_y',
       'Nametext_y', 'SpecifiedPerson_y', 'LineOneText_y', 'TypeCode_y',
       'ClassificationCode_y'],
      dtype='object')

In [30]:
# match 안되는 row 없음
new_df3['Nametext_y'].isnull().sum()

0

In [33]:
len(new_df3['상호'].unique()) # 53개의 상호

53

In [34]:
name = new_df3['상호'].unique()

info_name = set(info_new['Nametext'].unique()) # 상호명 info_name에 저장
for n in name:
    if n not in info_name:
        print(n) # 상호명 모두 match 됌

In [35]:
new_df3[new_df3['상호'] == '(주)케이원테크'].iloc[1]

상호                                      (주)케이원테크
업태                                           제조업
종목                                       반도체장비제작
일자                                         01-05
발급일자                                  2019-02-08
전송일자                                  2019-02-08
거래처                                 에스케이브로드밴드(주)
품명                        통신요금 UKEY-616939458609
공급가액                                       45000
세액                                          4500
상대계정                                         통신비
승인번호                    2019010550000066s8264514
ID_x                                         NaN
Nametext_x                                   NaN
SpecifiedPerson_x                            NaN
LineOneText_x                                NaN
TypeCode_x                                   NaN
ClassificationCode_x                         NaN
ID_y                                           0
Nametext_y                              (주)케이원테크
SpecifiedPerson_y   

In [36]:
new_df4 = new_df3[~new_df3['Nametext_x'].isnull()].copy()

In [39]:
len(new_df4['Nametext_x'].unique())

1913

In [40]:
new_df4.columns

Index(['상호', '업태', '종목', '일자', '발급일자', '전송일자', '거래처', '품명', '공급가액', '세액',
       '상대계정', '승인번호', 'ID_x', 'Nametext_x', 'SpecifiedPerson_x',
       'LineOneText_x', 'TypeCode_x', 'ClassificationCode_x', 'ID_y',
       'Nametext_y', 'SpecifiedPerson_y', 'LineOneText_y', 'TypeCode_y',
       'ClassificationCode_y'],
      dtype='object')

In [41]:
"""Index(['상호', '업태', '종목', '일자', '발급일자', '전송일자', '거래처', '품명', '공급가액', '세액',
'상대계정', '승인번호', 'ID_x', 'Nametext_x', 'SpecifiedPerson_x',
       'LineOneText_x', 'TypeCode_x', 'ClassificationCode_x', 'ID_y',
       'Nametext_y', 'SpecifiedPerson_y', 'LineOneText_y', 'TypeCode_y',
       'ClassificationCode_y'],
      dtype='object')""".replace("_y", "_invoicee").replace("_x", "_invoicer").replace("\n", "")

"Index(['상호', '업태', '종목', '일자', '발급일자', '전송일자', '거래처', '품명', '공급가액', '세액','상대계정', '승인번호', 'ID_invoicer', 'Nametext_invoicer', 'SpecifiedPerson_invoicer',       'LineOneText_invoicer', 'TypeCode_invoicer', 'ClassificationCode_invoicer', 'ID_invoicee',       'Nametext_invoicee', 'SpecifiedPerson_invoicee', 'LineOneText_invoicee', 'TypeCode_invoicee',       'ClassificationCode_invoicee'],      dtype='object')"

In [42]:
new_df4.columns = ['상호', '업태', '종목', '일자', '발급일자', '전송일자', '거래처', '품명', '공급가액', '세액','상대계정', '승인번호', 
                   'ID_invoicer', 'Nametext_invoicer', 'SpecifiedPerson_invoicer',   
                   'LineOneText_invoicer', 'TypeCode_invoicer', 'ClassificationCode_invoicer', 'ID_invoicee',   
                   'Nametext_invoicee', 'SpecifiedPerson_invoicee', 'LineOneText_invoicee', 'TypeCode_invoicee',       
                   'ClassificationCode_invoicee']



In [44]:
len(new_df4) # 전처리 완료된 Dataset

23781

# 최종본

In [438]:
new_df2

,상호,업태,종목,일자,발급일자,전송일자,거래처,품명,공급가액,세액,상대계정,승인번호,ID,Nametext,SpecifiedPerson,LineOneText,TypeCode,ClassificationCode
0,왓츠 주식회사,서비스/도소매/소,소프트웨어자문개발,01-01,2020-02-10,2020-02-10,(주)대륜이엔에스,도시가스 요금,60823,6082,수도광열비,2020010150000034f4zyptsp,2.178116e+09,(주)대륜이엔에스,전동수,서울시 노원구 덕릉로70길 20(상계동),제조도소매,도시가스제조및공급업
1,왓츠 주식회사,서비스/도소매/소,소프트웨어자문개발,01-06,2020-01-28,2020-01-30,주식회사 케이티,기가 인터넷 최대 1G (z!22572238130),45000,4500,통신비,2020010650000002j8538464,1.028143e+09,주식회사 케이티,황창규,경기도 성남시 분당구 불정로 90,통신업 부동산업,전신전화 부가통신 임대
2,왓츠 주식회사,서비스/도소매/소,소프트웨어자문개발,01-07,2020-01-07,2020-01-07,세빛조명,조명기구,88000,8800,소모품비,202001071000000014438566,6.042450e+09,세빛조명,정연순,서울특별시 강북구 인수봉로77길 29 1층(수유동),도매 및 소매업,조명기구
3,왓츠 주식회사,서비스/도소매/소,소프트웨어자문개발,01-08,2020-01-28,2020-01-30,주식회사 케이티,일반전화 (000204583123),3800,380,통신비,2020010850000002a1250084,1.028143e+09,주식회사 케이티,황창규,경기도 성남시 분당구 불정로 90,통신업 부동산업,전신전화 부가통신 임대
4,왓츠 주식회사,서비스/도소매/소,소프트웨어자문개발,01-15,2020-01-15,2020-01-16,원스토어 주식회사,12월 원스토어㈜ 상품 판매 서비스이용료,1200,120,지급수수료,202001154100017730001230,8.568600e+09,원스토어 주식회사,이재환,경기도 성남시 분당구 판교역로 146번길 20(백현동),서비스,부가통신
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26934,주식회사 씨앤세스시스템즈,"도소매,제조,전문과","전기,전자,반도체건",12-26,2019-12-26,2019-12-26,티와이이앤씨 주식회사,컨설팅(배기),3000000,300000,지급수수료,201912261000000091862024,8.758601e+09,티와이이앤씨 주식회사,탁영용,경기도 수원시 권선구 오목천로152번길 16 4층(고색동),도매 및 상품중개업,기타중개업
26935,주식회사 씨앤세스시스템즈,"도소매,제조,전문과","전기,전자,반도체건",12-26,2019-12-27,2019-12-28,특허법인뉴코리아,특허등록비용(10-2019-105231),1500000,150000,지급수수료,2019122641000115a144fs2x,4.228601e+09,특허법인뉴코리아,조인제,서울시 강남구 논현로87길 15-6 7층(역삼동),서비스,변리사외
26936,주식회사 씨앤세스시스템즈,"도소매,제조,전문과","전기,전자,반도체건",12-29,2019-12-29,2019-12-30,먹는샘물 경기남부총판,생수,17250,1725,복리후생비,20191229410000080006iglu,1.262523e+09,먹는샘물 경기남부총판,서상대,경기 이천시 백사면 이여로 282-14 4번창고(조읍리 608-2),도소매,생수.음료
26937,주식회사 씨앤세스시스템즈,"도소매,제조,전문과","전기,전자,반도체건",12-30,2019-12-30,2019-12-30,쌍마ENG,이중관제조설비 및 플랜지성형기 1차중도금,30000000,3000000,기계장치,201912301000000093899211,3.011880e+09,쌍마ENG,이구렬,충청북도 청주시 서원구 남이면 남석로 446,제조업,부품제작및가공


In [45]:
new_df4.to_csv("./0530합본.csv", index=False, encoding='utf-8-sig')

### 